#### Load the data

In [29]:
import pandas as pd
import pickle
import math
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score,mean_squared_log_error,median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import csv
import time

In [2]:
# pre
data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
training_data_list = load_obj("SBdata_all_feature-v2")
test_data = load_obj("SB_testData_all_feature-V2")

# Get the training_data_df
training_data_df = pd.DataFrame(training_data_list) 
final_training_data_df = training_data_df.iloc[:,2:20]
# 0.511756 0.028259
# get the labeled data df
final_labels_df = training_data_df.iloc[:,1]
test_data_df = pd.DataFrame(test_data) 

In [ ]:
# modeling
X=final_training_data_df

# get the data and label
y=final_labels_df

# training model
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)

# Gridsearch settings
rf = RandomForestRegressor()
X_train = X_t
y_train = y_t

pipeline = Pipeline([
       ('clf', RandomForestRegressor(criterion='mse'))
   ])
parameters = {
       'clf__n_estimators': (50,100),
       'clf__max_depth': (10, 30, 50),
       'clf__min_samples_split': (10, 50),
       'clf__min_samples_leaf': (10, 50)
}
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
   verbose=1, scoring='mse', cv=3)

grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [32]:
# save_obj(grid_search,'grid_search_rf_regressor_first_0_91')
print(type(parameters))

<class 'dict'>


In [30]:
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print('neg_mean_squared_error:', mean_squared_error(y_test, predictions))
print('mean_absolute_error:', mean_absolute_error(y_test, predictions))
print('mean_squared_error:', mean_squared_error(y_test, predictions))
print('mean_squared_log_error:', mean_squared_log_error(y_test, predictions))
print('median_absolute_error:', median_absolute_error(y_test, predictions))
print('explained_variance:', explained_variance_score(y_test, predictions))
print('r2_score:', r2_score(y_test, predictions))



Best score: 0.969
Best parameters set:
	clf__max_depth: 50
	clf__min_samples_leaf: 50
	clf__min_samples_split: 50
	clf__n_estimators: 100
neg_mean_squared_error: 0.06087385481307672
mean_absolute_error: 0.12373286679300308
mean_squared_error: 0.06087385481307672
mean_squared_log_error: 0.029958819272636803
median_absolute_error: 0.029153325173029987
explained_variance: 0.7565031815552645
r2_score: 0.7565029515468192


In [ ]:




def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    cnt=0
    for row in test_data_df.iterrows():
        row_df = pd.DataFrame(row[1]).T
        single_result = grid_search.predict(row_df)[0]
#         print(model.predict(row_df)[0][0])
        cnt+=1
        result.append((cnt, single_result))
        print(cnt)
    return result
result = predict()


'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Prediction']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
save_prediction_to_csv(result, "shawn_rf_regressor_small-v2")

# Save the result of the prediction

# Modelling

# Please save the training set as the csv file.

# Prediction

In [ ]:
result